In [ ]:
# Importação de bibliotecas
import tensorflow as tf
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# Diretórios para dados de treinamento, validação e teste
path = Path("Expressao facial")
train_data_dir = path/'train'
validation_data_dir = path/'val'
test_data_dir = path/'test'

In [ ]:
# Dimensões das imagens e número de classes
img_width, img_height = 512, 512
num_classes = 4

In [ ]:
# Hiperparâmetros
batch_size = 10 # Tamanho de imagens de lote de treinamento
epochs = 15 # Épocas

In [ ]:
# Rescale = redimensiona os valores dos pixels para o intervalo [0, 1]
# Cria uma instancia da classe ImageDataGnerator, para pré-processamento dos dados
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [ ]:
# Carregamento e pré-processamento dos dados de treinamento
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical', # Define que é multiclasses
    color_mode='grayscale'  # Definindo para imagens em escala de cinza
)

# Carregamento e pré-processamento dos dados de validação
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical', # Define que é multiclasses
    color_mode='grayscale'  # Definindo para imagens em escala de cinza
)

# Carregamento e pré-processamento dos dados de teste
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=False,
    class_mode='categorical', # Define que é multiclasses
    color_mode='grayscale'  # Definindo para imagens em escala de cinza
)

Found 5157 images belonging to 4 classes.
Found 2501 images belonging to 4 classes.
Found 2461 images belonging to 4 classes.


In [ ]:
# Definindo o modelo CNN
model = Sequential()
model.add(Conv2D(32,(5,5),activation="relu",input_shape=(img_width,img_height,1)))
model.add(Conv2D(32,(3,3),activation="relu"))
model.add(Conv2D(32,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64,(3,3),activation="relu"))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128,(3,3),activation="relu"))
model.add(Conv2D(128,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(256,(3,3),activation="relu"))
model.add(Conv2D(256,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(800,activation="relu"))
model.add(Dense(num_classes,activation="softmax"))

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Treinamento do modelo com validacao
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

In [ ]:
# Dicionario que mapeia as classes
class_indices = {
    'glioma_tumor': 0,
    'meningioma_tumor': 1,
    'no_tumor': 2,
    'pituitary_tumor': 3
}

In [ ]:
# Associa o dicionario ao gerador de teste
test_generator.class_indices = class_indices


In [ ]:
# Mapeia indices de classe para nomes de classe
class_names = {v: k for k, v in test_generator.class_indices.items()}

In [ ]:
# Avaliacao do modelo no conjunto de teste
y_true = test_generator.classes
y_pred = model.predict(test_generator)
y_pred_classes = [tf.argmax(x).numpy() for x in y_pred]

In [ ]:
# Calcula metricas
accuracy = accuracy_score(y_true, y_pred_classes)
confusion = confusion_matrix(y_true, y_pred_classes)
classification_rep = classification_report(y_true, y_pred_classes, target_names=class_names.values(), digits=4)

print(f'Test Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{confusion}')
print(f'Classification Report:\n{classification_rep}')

In [ ]:

# Salvando o modelo treinado
model.save('Tumor.h5')